In [ ]:
import sys
import os
from pathlib import Path
sys.path.append(os.path.abspath(".."))



In [2]:
import pandas as pd
from collections import defaultdict
import requests 
from bs4 import BeautifulSoup
import time
import pickle
from sqlalchemy import func
from sqlalchemy.inspection import inspect

from core import app, db
from models import *

DATA = Path('./data/version-3/')
DBDUMP = Path('data/db-dump/')

### Basic Stats

In [3]:
with app.app_context():
    print("Total Foods:", db.session.query(Food.food_id).distinct().count())
    print("Total Diseases:", db.session.query(Disease.disease_id).distinct().count())
    print("Total Genes:", db.session.query(Gene.gene_id).distinct().count())
    print("Total Chemicals:", db.session.query(Chemical.pubchem_id).distinct().count())


Total Foods: 6980
Total Diseases: 7610
Total Genes: 20558


C:\Users\Sneha\AppData\Local\Temp\ipykernel_9108\2356757312.py:2: SAWarning: relationship 'Disease.foods' will copy column disease.disease_id to column food_disease.disease_id, which conflicts with relationship(s): 'Food.diseases' (copies disease.disease_id to food_disease.disease_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="diseases"' to the 'Disease.foods' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated operation.)
  print("Total Foods:", db.session.query(Food.food_i

Total Chemicals: 6992


In [4]:
with app.app_context():
    cats = [v[0] for v in (db.session.query(Food.food_category).distinct().all())]

In [5]:
len(set([c for catlst in cats for c in catlst.split(', ')]))

25

In [6]:
with app.app_context():
    print("Total Foods:", len([val for val in db.session.query(Food_disease.food_id).distinct()]))
    print("Total Foods:", len([val for val in db.session.query(Food_gene.food_id).distinct()]))
    print("Total Foods:", len([val for val in db.session.query(Food_chemical.food_id).distinct()]))

    print("Total Diseases:", len([val for val in db.session.query(Disease.disease_id).distinct()]))
    print("Total Genes:", len([val for val in db.session.query(Gene.gene_id).distinct()]))
    print("Total Chemicals:", len([val for val in db.session.query(Chemical.pubchem_id).distinct()]))


Total Foods: 2337
Total Foods: 2159
Total Foods: 1491
Total Diseases: 7610
Total Genes: 20558
Total Chemicals: 6992


In [7]:
with app.app_context():
    count = db.session.query(
        func.count()
    ).select_from(
        db.session.query(Food_disease.food_id, Food_disease.disease_id).distinct().subquery()
    ).scalar()

    print("Unique (food_id, disease_id) pairs:", count)


Unique (food_id, disease_id) pairs: 280328


In [8]:
with app.app_context():
    count = db.session.query(
        func.count()
    ).select_from(
        db.session.query(Food_chemical.food_id, Food_chemical.pubchem_id).distinct().subquery()
    ).scalar()

    print("Unique (food_id, pubchem_id) pairs:", count)

Unique (food_id, pubchem_id) pairs: 66577


In [9]:
with app.app_context():
    count = db.session.query(
        func.count()
    ).select_from(
        db.session.query(Food_gene.food_id, Food_gene.gene_id).distinct().subquery()
    ).scalar()

    print("Unique (food_id, gene_id) pairs:", count)

Unique (food_id, gene_id) pairs: 3792433


In [10]:
with app.app_context():
    count = db.session.query(
        func.count()
    ).select_from(
        db.session.query(Disease_gene.disease_id, Disease_gene.gene_id).distinct().subquery()
    ).scalar()

    print("Unique (disease_id, gene_id) pairs:", count)

Unique (disease_id, gene_id) pairs: 5150217


In [11]:
with app.app_context():
    count = db.session.query(
        func.count()
    ).select_from(
        db.session.query(Chemical_disease.pubchem_id, Chemical_disease.disease_id).distinct().subquery()
    ).scalar()

    print("Unique (pubchem_id, disease_id) pairs:", count)

Unique (pubchem_id, disease_id) pairs: 284612


In [12]:
with app.app_context():
    count = db.session.query(
        func.count()
    ).select_from(
        db.session.query(Chemical_gene.pubchem_id, Chemical_gene.gene_id).distinct().subquery()
    ).scalar()

    print("Unique (pubchem_id, gene_id) pairs:", count)

Unique (pubchem_id, gene_id) pairs: 581465


### Utility functions

In [13]:
def clean_colnames(df):
    df.columns = [col.replace(' ', '-').lower() for col in df.columns]

In [14]:
def get_primary_keys(model):
    return [key.name for key in inspect(model).primary_key]

In [15]:
def add_to_db(obj, dl):
    primary_keys = get_primary_keys(obj)

    for i, item in enumerate(dl):
        filter_kwargs = {key: item[key] for key in primary_keys if key in item}
        existing = obj.query.filter_by(**filter_kwargs).first()

        if existing:
            continue  # Skip if already exists

        db.session.add(obj(**item))
        if (i + 1) % 5000 == 0:
            db.session.commit()

    db.session.commit()


In [ ]:
def add_to_db_df(obj, df):
     primary_keys = get_primary_keys(obj)

     for i, row in df.iterrows():
         item = row.to_dict()
         filter_kwargs = {key: item[key] for key in primary_keys if key in item}
         existing = obj.query.filter_by(**filter_kwargs).first()

         if existing:
             continue  # Skip if already exists

         db.session.add(obj(**item))
         if (i + 1) % 5000 == 0:
             db.session.commit()
     db.session.commit()

     
#  import time
# from sqlalchemy.exc import OperationalError

# def add_to_db_df(obj, df, batch_size=500, max_retries=5, retry_wait=0.5):
#     primary_keys = get_primary_keys(obj)

#     for i, row in enumerate(df.iterrows()):
#         item = row[1].to_dict()  # row is (index, Series)
#         filter_kwargs = {key: item[key] for key in primary_keys if key in item}
#         existing = obj.query.filter_by(**filter_kwargs).first()

#         if existing:
#             continue  # Skip if already exists

#         db.session.add(obj(**item))

#         # Commit in batches
#         if (i + 1) % batch_size == 0:
#             for attempt in range(max_retries):
#                 try:
#                     db.session.commit()
#                     break
#                 except OperationalError as e:
#                     db.session.rollback()
#                     print(f"[Batch Commit Error] Attempt {attempt + 1} of {max_retries}: {e}")
#                     time.sleep(retry_wait * (attempt + 1))
#             else:
#                 print(f"[Batch Commit Failed Permanently] Skipping batch ending at row {i + 1}")

#     # Final commit
#     try:
#         db.session.commit()
#     except OperationalError as e:
#         db.session.rollback()
#         print(f"[Final Commit Error] {e}")



In [17]:
def remove_from_db(obj):
    obj.query.delete()
    db.session.commit()

In [18]:
# remove_from_db(Food_disease)
# remove_from_db(Food_chemical)
# remove_from_db(Food_gene)
# remove_from_db(Disease_gene)
# remove_from_db(Chemical_disease)
# remove_from_db(Chemical_gene)
# remove_from_db(Food)
# remove_from_db(Disease)
# remove_from_db(Gene)
# remove_from_db(References)
# remove_from_db(Chemical)

## Vanilla Associations

#### a. Food-Disease

In [19]:
food_dis = pd.read_csv(DATA/'food-disease.tsv', sep='\t', encoding='utf-8')
food_dis.drop_duplicates(['food-id', 'disease-id', 'association', 'pmid'], inplace=True)
food_dis = food_dis.where((pd.notnull(food_dis)), None)
food_dis['association'] = food_dis['association'].apply(str.lower)
food_dis = food_dis.loc[:, ['food-id', 'disease-id', 'association', 'pmid']]
food_dis['pmid'] = food_dis['pmid'].apply(str)
food2dis = defaultdict(lambda: set(),
                       food_dis.groupby('food-id').agg(lambda s: set(s))['disease-id'].to_dict())
food_dis.head(1)

,food-id,disease-id,association,pmid
0,Plant ID:4045,MESH:D004487,negative,7398283


In [20]:
food_dis = food_dis[food_dis['food-id'].map(lambda s: 'Seafood' not in s)]

In [21]:
food_dis.drop_duplicates(['food-id', 'disease-id', 'association']).shape

(21207, 4)

In [22]:
len(set(food_dis['food-id']))

1781

#### b. Food-Chemical

In [23]:
food_chem = pd.read_csv(DATA/'food-chemical.tsv', sep='\t', encoding='utf-8')
food2chem = defaultdict(lambda: set(),
                        food_chem.groupby('food-id').agg(lambda s: set(s))['pubchem-id'].to_dict())

food_chem.head(1)

,food-id,pubchem-id,content,reference
0,Plant ID:100170,5280378,Detected but not quantified,KNApSAcK


In [24]:
food_chem.shape

(66577, 4)

#### c. Gene-Disease

In [25]:
gene_dis = pd.read_csv(DATA/'gene-disease.tsv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')
dis2gene = defaultdict(lambda: set(),
                        gene_dis.groupby('disease-id').agg(lambda s: set(s))['gene-id'].to_dict())
gene2dis = defaultdict(lambda: set(),
                        gene_dis.groupby('gene-id').agg(lambda s: set(s))['disease-id'].to_dict())

gene_dis.head()

,disease-id,gene-id,source
0,MESH:C000591739,1588,ORPHANET
1,MESH:C000596385,26504,UNIPROT
2,MESH:C000598645,9499,CTD_human;ORPHANET;UNIPROT
3,MESH:C000600608,29940,ORPHANET
4,MESH:C000600608,113189,CTD_human;ORPHANET;UNIPROT


#### d. Chemical-Disease

In [26]:
chem_dis = pd.read_csv(DATA/'chemical-disease.tsv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')
chem_dis = chem_dis.groupby(['pubchem-id', 'disease-id']).agg(lambda s: sorted(s)[-1]).reset_index()
dis2chem = defaultdict(lambda: set(),
                       chem_dis.groupby('disease-id').agg(lambda s: set(s))['pubchem-id'].to_dict())
chem2dis = defaultdict(lambda: set(),
                       chem_dis.groupby('pubchem-id').agg(lambda s: set(s))['disease-id'].to_dict())

chem_dis.head(1)

,pubchem-id,disease-id,type
0,19,MESH:D007674,therapeutic


#### e. Chemical-Gene

In [27]:
chem_gene = pd.read_csv(DATA/'chemical-gene.tsv', sep='\t', encoding='utf-8')
chem2gene = defaultdict(lambda: set(),
                        chem_gene.groupby('pubchem-id').agg(lambda s: set(s))['gene-id'].to_dict())
gene2chem = defaultdict(lambda: set(),
                        chem_gene.groupby('gene-id').agg(lambda s: set(s))['pubchem-id'].to_dict())
chem_gene.head()

,pubchem-id,gene-id,interaction-actions,source
0,19,351,affects^binding|decreases^reaction,CTD
1,19,4313,decreases^activity|decreases^reaction|decrease...,CTD
2,51,3091,affects^binding|affects^cotreatment|increases^...,CTD
3,51,7428,affects^binding|affects^cotreatment|increases^...,CTD
4,51,8290,affects^cotreatment|decreases^methylation|incr...,CTD


## Lexicons

### A. Food 

In [28]:
# Read file
foodlex = pd.read_csv(DATA/'food-lexicon-automated.tsv', sep='\t', encoding='utf-8')
clean_colnames(foodlex)

del foodlex['foodb-group'], foodlex['foodb-subgroup']

# Replace NaN with None
foodlex = foodlex.where((pd.notnull(foodlex)), None)

# Rename columns to match db schema.
foodlex.rename(columns={
    'common-names': 'common_names',
    'food-name': 'display_name',
    'food-id': 'food_id',
    'food-category': 'food_category',
    'scientific-name': 'scientific_name',
    'tax-id':'tax_id',
}, inplace=True)

foodlex.head()

,common_names,food_id,food_category,display_name,scientific_name,tax_id
0,alexanders; horse parsley,Plant ID:40962,Miscellaneous,Alexanders,Smyrnium olusatrum,40962.0
1,None,Plant ID:942083,Miscellaneous,Scrophularia umbrosa,Scrophularia umbrosa,942083.0
2,dotted blazing star,Plant ID:344074,Miscellaneous,Dotted blazing star,Liatris punctata,344074.0
3,plymouth pear,Plant ID:761867,Miscellaneous,Plymouth pear,Pyrus cordata,761867.0
4,None,Plant ID:49166,Miscellaneous,Rhododendron kaempferi,Rhododendron kaempferi,49166.0


In [29]:
# Redundancy check
assert len((set(food_dis['food-id']) | set(food_chem['food-id'])) - set(foodlex['food_id'])) == 0,\
    'Food lexicon is not exhaustive!'

In [30]:
# Dump to disk
foodlex.to_csv(DBDUMP/'food-lexicon.tsv', sep='\t', encoding='utf-8', index=None)

In [31]:
with app.app_context():
    data_list= list(foodlex.T.to_dict().values())
    add_to_db(Food, data_list)
    del data_list

### B. Disease

In [32]:
dislex = pd.read_csv(DATA / 'CTD_diseases_new.tsv', sep='\t', encoding='utf-8', on_bad_lines='skip')

# Keep only selected columns.
dislex = dislex[['DiseaseID', 'DiseaseName', 'SlimMappings', 'Synonyms']]

# Replace nan with None
dislex = dislex.where((pd.notnull(dislex)), None)

# Rename to match schema
dislex.rename(columns={
    'DiseaseID': 'disease_id',
    'DiseaseName': 'disease_name',
    'SlimMappings': 'disease_category',
    'Synonyms': 'disease_synonyms'
}, inplace=True)

dislex.head()

,disease_id,disease_name,disease_category,disease_synonyms
0,MESH:C538288,10p Deletion Syndrome (Partial),Congenital abnormality|Genetic disease (inborn...,"Chromosome 10, 10p- Partial|Chromosome 10, mon..."
1,MESH:C535484,13q deletion syndrome,Congenital abnormality|Genetic disease (inborn...,Chromosome 13q deletion|Chromosome 13q deletio...
2,MESH:C579849,15q24 Microdeletion,Congenital abnormality|Genetic disease (inborn...,15q24 Deletion|15q24 Microdeletion Syndrome|In...
3,MESH:C579850,16p11.2 Deletion Syndrome,Congenital abnormality|Genetic disease (inborn...,None
4,MESH:C567076,"17,20-Lyase Deficiency, Isolated",Congenital abnormality|Endocrine system diseas...,"17-Alpha-Hydroxylase-17,20-Lyase Deficiency, C..."


In [33]:
# Redundancy check
assert len((set(food_dis['disease-id']) | set(chem_dis['disease-id']) | set(gene_dis['disease-id'])) 
           - set(dislex['disease_id'])) == 0,\
    'Disease lexicon is not exhaustive!'

# Subset disease lexicon
dislex = dislex.set_index('disease_id').\
    loc[list(set(food_dis['disease-id']) | set(chem_dis['disease-id']) | set(gene_dis['disease-id']))].\
    reset_index()

In [34]:
dislex.to_csv(DBDUMP/'disease-lexicon.tsv', sep='\t', encoding='utf-8', index=None)

In [35]:
with app.app_context():
    add_to_db_df(Disease, dislex)

### C. Chemical

In [36]:
chem = pd.read_csv(DATA/'chemical-lexicon.tsv', sep='\t', encoding='utf-8').drop_duplicates(['pubchem_id'])

# Replace nan with null values
chem = chem.where((pd.notnull(chem)), None)

# Rename columns
chem.rename(columns={'canonical_smiles': 'smiles',},inplace=True)

del chem['inchi'], chem['inchikey']

In [37]:
# Redundancy check
assert len((set(food_chem['pubchem-id']) | set(chem_dis['pubchem-id']) | set(chem_gene['pubchem-id'])) 
           - set(chem['pubchem_id'])) == 0,\
    'Chemical lexicon is not exhaustive!'

In [38]:
chem.to_csv(DBDUMP/'chemical-lexicon.tsv', sep='\t', encoding='utf-8', index=None)

In [39]:
with app.app_context():
    for i in range(0, len(chem), 50000):
        data_list = list(chem.loc[i:50000-1+i, :].T.to_dict().values())
        add_to_db(Chemical, data_list)
        print(i)
    print('done')

0
done


### D. Gene

In [40]:
from Bio import Entrez
# pip install biopython

Entrez.email = "rujhil24076@iiitd.ac.in"

def retrieve_annotation(id_list):

    """Annotates Entrez Gene IDs using Bio.Entrez, in particular epost (to
    submit the data to NCBI) and esummary to retrieve the information.
    Returns a list of dictionaries with the annotations."""

    request = Entrez.epost("gene",id=",".join(id_list))
    result = Entrez.read(request)

    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db="gene", webenv=webEnv, query_key =
            queryKey)
    annotations = Entrez.read(data)

    return annotations

records = list()
covered = list()

In [41]:
import time
import pickle
from http.client import IncompleteRead

with app.app_context():
    genes = [str(g[0]) for g in db.session.query(Gene.gene_id).distinct().all()]

for i in range(0, len(genes), 100):
    if i in covered:
        continue

    try:
        batch = genes[i: i + 100]
        annotation = retrieve_annotation(batch)
        records += annotation['DocumentSummarySet']['DocumentSummary']
        covered.append(i)

        if i % 1000 == 0:
            print("Covered:", len(records))
        
        time.sleep(0.4)  # NCBI recommends max 3 requests per second

    except IncompleteRead as e:
        print(f"IncompleteRead error at batch {i}, skipping.")
        continue

    except Exception as e:
        print(f"Error at batch {i}: {e}")
        time.sleep(5)  # wait a bit longer before retrying next
        continue

# Final record processing
records = [dict(d) for d in records]
for rec, geneid in zip(records, genes):
    rec['gene_id'] = geneid

# Save results
pickle.dump(records, open('data/version-3/gene-lexicon-raw.p', 'wb'))


Covered: 100
Covered: 1100
Covered: 2100
Covered: 3100
Covered: 4100
Covered: 5100
Covered: 6100
Covered: 7100
Covered: 8100
Covered: 9100
Covered: 10100
Covered: 11100
Covered: 12100
Covered: 13100
Covered: 14100
Covered: 15100
Covered: 16100
Covered: 17100
Covered: 18100
Covered: 19100
Covered: 20100


In [42]:
records = pickle.load(open('./data/version-3/gene-lexicon-raw.p', 'rb'))
genelex = pd.DataFrame(records)

# Subset
genelex = genelex[['gene_id', 'Description', 'Name', 'Organism',
                   'OtherAliases', 'OtherDesignations']]

# Keep scientific name of organism only
genelex['Organism'] = genelex['Organism'].map(lambda s: s['ScientificName'])

# Rename columns
genelex.rename(columns={
    'Description': 'gene_name',
    'Name': 'gene_symbol',
    'Organism': 'organism',
    'OtherAliases': 'other_symbols',
    'OtherDesignations': 'synonyms'
}, inplace=True)

del records

genelex.head(2)

,gene_id,gene_name,gene_symbol,organism,other_symbols,synonyms
0,1,alpha-1-B glycoprotein,A1BG,Homo sapiens,"A1B, ABG, GAB, HYST2477",alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...
1,10,N-acetyltransferase 2,NAT2,Homo sapiens,"AAC2, NAT-2, PNAT",arylamine N-acetyltransferase 2|N-acetyltransf...


In [43]:
# Count how many were annotated
print("Annotated gene IDs:", len(set(genelex['gene_id'])))

# Count how many are in chemical-gene and gene-disease data
all_needed = set(chem_gene['gene-id']) | set(gene_dis['gene-id'])
print("Total required gene IDs:", len(all_needed))

# What fraction is missing?
missing = all_needed - set(genelex['gene_id'])
print("Missing gene IDs:", len(missing), f"({100 * len(missing)/len(all_needed):.2f}%)")

# Example missing gene IDs
print("Examples:", list(missing)[:10])


Annotated gene IDs: 20558
Total required gene IDs: 20550
Missing gene IDs: 20550 (100.00%)
Examples: [1, 2, 3, 131076, 9, 10, 12, 13, 14, 15]


In [44]:
# If loaded from pickle
import pickle
genelex = pd.DataFrame(pickle.load(open('data/version-3/gene-lexicon-raw.p', 'rb')))

# Ensure all gene IDs are strings (to match other DataFrames)
genelex['gene_id'] = genelex['gene_id'].astype(str)


In [45]:
chem_gene['gene-id'] = chem_gene['gene-id'].astype(str)
gene_dis['gene-id'] = gene_dis['gene-id'].astype(str)

In [46]:
all_needed_genes = set(chem_gene['gene-id']) | set(gene_dis['gene-id'])
available_genes = set(genelex['gene_id'])

missing = all_needed_genes - available_genes

print(f"❌ Still missing: {len(missing)} gene IDs")
print(f"Example missing IDs: {list(missing)[:10]}")

❌ Still missing: 0 gene IDs
Example missing IDs: []


In [47]:
import time
from Bio import Entrez
from http.client import IncompleteRead
import pickle
import pandas as pd

Entrez.email = "rujhil24076@iiitd.ac.in"  # Update with your real email

# Load current genelex and missing list
genelex = pd.DataFrame(pickle.load(open('data/version-3/gene-lexicon-raw.p', 'rb')))
genelex['gene_id'] = genelex['gene_id'].astype(str)

all_needed_genes = set(chem_gene['gene-id'].astype(str)) | set(gene_dis['gene-id'].astype(str))
existing_genes = set(genelex['gene_id'])
still_missing = sorted(list(all_needed_genes - existing_genes))

print(f"Retrying {len(still_missing)} missing gene IDs...")

recovered = []

for i in range(0, len(still_missing), 10):
    batch = still_missing[i:i+10]

    for attempt in range(3):  # up to 3 retries per batch
        try:
            annotation = retrieve_annotation(batch)
            summaries = annotation['DocumentSummarySet']['DocumentSummary']

            for rec, gid in zip(summaries, batch):
                d = dict(rec)
                d['gene_id'] = gid
                recovered.append(d)

            print(f"✅ Batch {i}-{i+10} recovered, total: {len(recovered)}")
            time.sleep(0.4)
            break  # exit retry loop

        except IncompleteRead:
            print(f"⚠️ IncompleteRead at batch {i}-{i+10}, retrying...")
            time.sleep(2 * (attempt + 1))  # backoff
        except Exception as e:
            print(f"❌ Error at batch {i}-{i+10}: {e}")
            time.sleep(2 * (attempt + 1))

# Merge with existing and save
recovered_df = pd.DataFrame(recovered)
genelex = pd.concat([genelex, recovered_df], ignore_index=True)
genelex = genelex.drop_duplicates(subset=['gene_id'])

pickle.dump(genelex.to_dict(orient='records'), open('data/version-3/gene-lexicon-full.p', 'wb'))

print(f"✅ Final genelex now has {len(genelex)} entries.")


Retrying 0 missing gene IDs...
✅ Final genelex now has 20558 entries.


In [48]:
genelex.to_csv(DBDUMP/'gene-lexicon.tsv', sep='\t', encoding='utf-8', index=None)

In [49]:
print(genelex.columns)


Index(['Name', 'Description', 'Status', 'CurrentID', 'Chromosome',
       'GeneticSource', 'MapLocation', 'OtherAliases', 'OtherDesignations',
       'NomenclatureSymbol', 'NomenclatureName', 'NomenclatureStatus', 'Mim',
       'GenomicInfo', 'GeneWeight', 'Summary', 'ChrSort', 'ChrStart',
       'Organism', 'LocationHist', 'gene_id'],
      dtype='object')


In [50]:
genelex_renamed = genelex.rename(columns={
    'Name': 'gene_name',
    'NomenclatureSymbol': 'gene_symbol',
    'Organism': 'organism',
    'OtherAliases': 'other_symbols',
    'OtherDesignations': 'synonyms'
})

In [51]:
valid_cols = ['gene_id', 'gene_name', 'gene_symbol', 'organism', 'other_symbols', 'synonyms']

# Only keep the columns that are now present
available_cols = [col for col in valid_cols if col in genelex_renamed.columns]

cleaned_genelex = genelex_renamed[available_cols].copy()

# Fill missing columns with None
for col in valid_cols:
    if col not in cleaned_genelex.columns:
        cleaned_genelex[col] = None


In [52]:
cleaned_genelex['organism'] = cleaned_genelex['organism'].apply(
    lambda org: org.get('ScientificName') if isinstance(org, dict) else org
)

with app.app_context():
    add_to_db_df(Gene, cleaned_genelex)

### E. References

In [53]:
references = pd.read_csv(DATA/'publication-records.tsv', sep='\t', encoding='utf-8')
clean_colnames(references)

references = references.where((pd.notnull(references)), None)

references.rename(columns={
    'authors': 'authors',
    'journal-name': 'journal_name',
    'journal-name-abbrv.': 'journal_name_abbr',
    'publication-type':'publication_type',
},inplace=True)

pmids_not_mapped = list(str(s) for s in set(food_dis['pmid'].apply(int)) - set(references['pmid']))
references = references.set_index('pmid').\
    reindex(index=references.pmid.tolist() + pmids_not_mapped).fillna('').\
    reset_index()

references.head()

,pmid,authors,date,journal_name,journal_name_abbr,publication_type,title
0,17344941,Elkiran T|Harputluoglu H|Yasar U|Babaoglu MO|D...,2007 Jan-Feb,Methods and findings in experimental and clini...,Methods Find Exp Clin Pharmacol,Clinical Trial|Journal Article,Differential alteration of drug-metabolizing e...
1,25456153,Severins N|Mensink RP|Plat J,2015 Feb,"Nutrition, metabolism, and cardiovascular dise...",Nutr Metab Cardiovasc Dis,Journal Article|Randomized Controlled Trial,Effects of lutein-enriched egg yolk in butterm...
2,23922960,Li C|Zuo C|Deng G|Kuang R|Yang Q|Hu C|Sheng O|...,2013,PloS one,PLoS One,Journal Article,Contamination of bananas with beauvericin and ...
3,8766742,Güneser S|Atici A|Cengizler I|Alparslan N,1996 May-Jun,Allergologia et immunopathologia,Allergol Immunopathol (Madr),Journal Article,Inhalant allergens: as a cause of respiratory ...
4,19918387,Brvar M|Bunc M,2009 Sep 9,Cases journal,Cases J,Journal Article,High-degree atrioventricular block in acute et...


In [54]:
references.to_csv(DBDUMP/'references.tsv', sep='\t', encoding='utf-8', index=None)

In [55]:
with app.app_context():
    add_to_db_df(References, references)

## Final Associations

### Food-Disease

In [56]:
# Curated
neg_fd = food_dis[food_dis.association == 'negative'].\
    groupby(['food-id', 'disease-id']).agg(lambda s: '|'.join(s))[['pmid']].reset_index(())
neg_fd.reset_index(inplace=True)
neg_fd.rename(columns={'pmid': 'negative-pmid'}, inplace=True)

pos_fd = food_dis[food_dis.association == 'positive'].\
    groupby(['food-id', 'disease-id']).agg(lambda s: '|'.join(s))[['pmid']]
pos_fd.reset_index(inplace=True)
pos_fd.rename(columns={'pmid': 'positive-pmid'}, inplace=True)

food_dis_curated = pos_fd.merge(neg_fd, on=['food-id', 'disease-id'], how='outer').fillna('')

# Inferred
fd_inferred = list()
for food, chemicals in food2chem.items():
    for chem in chemicals:
        for dis in chem2dis[chem]:
            fd_inferred.append([food, dis, str(chem)])
            
fd_inferred = pd.DataFrame(fd_inferred, 
                           columns=['food-id', 'disease-id', 'pubchem-id'])
fd_inferred = fd_inferred.groupby(['food-id', 'disease-id']).agg(lambda s: '|'.join(s)).reset_index()
del pos_fd, neg_fd

fd_inferred.head(2)

,food-id,disease-id,pubchem-id
0,Plant ID:100170,MESH:D002583,5280378
1,Plant ID:100170,MESH:D058186,5280378


In [57]:
food_dis_db = food_dis_curated.merge(fd_inferred, on=['food-id', 'disease-id'], how='outer').fillna('')
food_dis_db['weight'] = food_dis_db.apply(lambda row: len(row['positive-pmid'].split('|')) +
                                          len(row['negative-pmid'].split('|')), 1)
del fd_inferred, food_dis_curated

food_dis_db = food_dis_db.where((pd.notnull(food_dis_db)), None)

food_dis_db.rename(columns={
    'food-id': 'food_id',
    'disease-id': 'disease_id',
    'positive-pmid': 'positive_pmid',
    'negative-pmid': 'negative_pmid',
    'pubchem-id': 'pubchem_id',
    'weight':'weight',
}, inplace=True)

food_dis_db.head()

,food_id,disease_id,positive_pmid,negative_pmid,pubchem_id,weight
0,AlcoholicBev ID:17,MESH:D001523,7195588,,,2
1,AlcoholicBev ID:17,MESH:D003866,6798614,,,2
2,AlcoholicBev ID:17,MESH:D004487,,27873566,,2
3,AlcoholicBev ID:17,MESH:D011041,,11639831,,2
4,AlcoholicBev ID:18,MESH:D001249,,6639863,,2


In [58]:
food_dis_db.to_csv(DBDUMP/'food-disease.tsv', sep='\t', encoding='utf-8', index=None)

In [59]:
with app.app_context():
    add_to_db_df(Food_disease, food_dis_db)

### Food-chemical

In [60]:
food_chem_db = food_chem.where((pd.notnull(food_chem)), None)

food_chem_db = food_chem_db.where((pd.notnull(food_chem_db)), None)

food_chem_db.rename(columns={
    'pubchem-id': 'pubchem_id',
    'food-id': 'food_id',
    'reference': 'references',
    'type':'type_relation',
    'inference-network':'inference_network'
}, inplace=True)

food_chem_db.head()

,food_id,pubchem_id,content,references
0,Plant ID:100170,5280378,Detected but not quantified,KNApSAcK
1,Plant ID:100170,44260113,Detected but not quantified,KNApSAcK
2,Plant ID:100506,1150,Detected but not quantified,KNApSAcK
3,Plant ID:100506,5202,Detected but not quantified,KNApSAcK
4,Plant ID:100506,5372945,Detected but not quantified,KNApSAcK


In [61]:
food_chem_db.to_csv(DBDUMP/'food-chemical.tsv', sep='\t', encoding='utf-8', index=None)

In [62]:
with app.app_context():
    add_to_db_df(Food_chemical, food_chem_db)

### Food-gene

In [113]:
# --- Chemical inference aggregation ---
fg_inf_chem = []
for food, chemlst in food2chem.items():
    for chem in chemlst:
        for gene in chem2gene[chem]:
            fg_inf_chem.append((food, gene, chem))

df_chem = pd.DataFrame(fg_inf_chem, columns=['food_id', 'gene_id', 'pubchem_id'])
df_chem['gene_id'] = df_chem['gene_id'].astype(str)
df_chem['food_id'] = df_chem['food_id'].astype(str)

# Convert pubchem_id to str during aggregation
df_chem = df_chem.groupby(['food_id', 'gene_id'])['pubchem_id'].apply(
    lambda x: '|'.join(sorted(set(str(i) for i in x)))
).reset_index()
df_chem.rename(columns={'pubchem_id': 'via_chemicals'}, inplace=True)


In [115]:
# --- Disease inference aggregation ---
fg_inf_dis = []
for food, dislst in food2dis.items():
    for dis in dislst:
        for gene in dis2gene[dis]:
            fg_inf_dis.append((food, gene, dis))

df_dis = pd.DataFrame(fg_inf_dis, columns=['food_id', 'gene_id', 'disease_id'])
df_dis['gene_id'] = df_dis['gene_id'].astype(str)
df_dis['food_id'] = df_dis['food_id'].astype(str)

df_dis = df_dis.groupby(['food_id', 'gene_id'])['disease_id'].apply(
    lambda x: '|'.join(sorted(set(x)))
).reset_index()
df_dis.rename(columns={'disease_id': 'via_diseases'}, inplace=True)

In [ ]:
#fg_inf_chem.head()
fg_inf_dis.head().to_dict()


{'food-id': {0: 'AlcoholicBev ID:17',
  1: 'AlcoholicBev ID:17',
  2: 'AlcoholicBev ID:17',
  3: 'AlcoholicBev ID:17',
  4: 'AlcoholicBev ID:17'},
 'gene-id': {0: 2, 1: 15, 2: 52, 3: 81, 4: 100},
 'disease-id': {0: 'MESH:D003866',
  1: 'MESH:D003866',
  2: 'MESH:D003866',
  3: 'MESH:D004487',
  4: 'MESH:D004487'}}

In [120]:
# Make sure everything is string
df_dis['gene_id'] = df_dis['gene_id'].astype(str)
df_dis['food_id'] = df_dis['food_id'].astype(str)
df_chem['gene_id'] = df_chem['gene_id'].astype(str)
df_chem['food_id'] = df_chem['food_id'].astype(str)

# Merge with full outer join on common keys
food_gene_db = pd.merge(df_dis, df_chem, on=['food_id', 'gene_id'], how='outer')

# Fill NaN with empty string for merge consistency
food_gene_db['via_diseases'] = food_gene_db['via_diseases'].fillna('')
food_gene_db['via_chemicals'] = food_gene_db['via_chemicals'].fillna('')

# Deduplicate and rejoin in case of multiple entries per key
food_gene_db = food_gene_db.groupby(['food_id', 'gene_id']).agg({
    'via_diseases': lambda x: '|'.join(sorted(set(i for i in x if i))),
    'via_chemicals': lambda x: '|'.join(sorted(set(i for i in x if i)))
}).reset_index()

# Final weight column (correct parentheses)
food_gene_db['weight'] = food_gene_db.apply(
    lambda row:
        (len(row['via_diseases'].split('|')) if row['via_diseases'] else 0) +
        (len(row['via_chemicals'].split('|')) if row['via_chemicals'] else 0),
    axis=1
)


In [123]:
food_gene_db['weight'] = food_gene_db.apply(
    lambda row: 
        (len(str(row['via_diseases']).split('|')) if row['via_diseases'] else 0) +
        (len(str(row['via_chemicals']).split('|')) if row['via_chemicals'] else 0),
    axis=1
)

In [124]:
food_gene_db.to_csv(DBDUMP/'food-gene.tsv', sep='\t', encoding='utf-8', index=None)

In [128]:
def add_to_db_df_fg(obj, df):
    df = df.where(pd.notnull(df), None)

    # Get model's valid columns
    model_columns = {c.name for c in obj.__table__.columns}
    df = df[[col for col in df.columns if col in model_columns]]  # Drop extras like 'weight'

    for i, row in df.iterrows():
        item = row.to_dict()
        db.session.merge(obj(**item))

        if (i + 1) % 5000 == 0:
            db.session.commit()

    db.session.commit()


In [129]:
with app.app_context():
# del food_gene_db['weight']
    add_to_db_df_fg(Food_gene, food_gene_db)

### Disease Gene

In [ ]:
# Curated
dg_curated = gene_dis.copy(deep=True)

# Inferred through chemicals
dg_inferred = list()
for disease, chemlst in dis2chem.items():
    for chem in chemlst:
        for gene in chem2gene[chem]:
            dg_inferred.append([disease, str(chem), gene])
            
dg_inferred = pd.DataFrame(dg_inferred, columns=['disease-id', 'pubchem-id', 'gene-id'])
dg_inferred = dg_inferred.groupby(['disease-id', 'gene-id']).agg(lambda s: '|'.join(s)).reset_index()
dg_inferred.head(2)

,disease-id,gene-id,pubchem-id
0,MESH:C531617,14,14985
1,MESH:C531617,35,14985


In [ ]:
dg_curated['gene-id'] = dg_curated['gene-id'].astype(str)
dg_inferred['gene-id'] = dg_inferred['gene-id'].astype(str)
dg_curated['disease-id'] = dg_curated['disease-id'].astype(str)
dg_inferred['disease-id'] = dg_inferred['disease-id'].astype(str)

In [ ]:
disease_gene_db = dg_curated.merge(dg_inferred, on=['disease-id', 'gene-id'], how='outer').fillna('')

In [ ]:
disease_gene_db.rename(columns={'pubchem-id': 'via-chemicals'}, inplace=True)
del dg_inferred, dg_curated

disease_gene_db = disease_gene_db.where((pd.notnull(disease_gene_db)), None)

disease_gene_db.rename(columns={
    'gene-id': 'gene_id',
    'disease-id': 'disease_id',
    'via-chemicals': 'via_chemicals',
    'source': 'reference'
}, inplace=True)


In [ ]:
disease_gene_db.to_csv(DBDUMP/'disease-gene.tsv', sep='\t', encoding='utf-8', index=None)

In [ ]:
with app.app_context():
    add_to_db_df(Disease_gene, disease_gene_db)

### Disease chemical

In [ ]:
# Curated
chem_dis_curated = chem_dis.copy(deep=True)
chem_dis_curated['pubchem-id'] = chem_dis_curated['pubchem-id'].apply(str)

# Inferred through genes
dc_inferred = list()
for disease, genelst in dis2gene.items():
    for gene in genelst:
        for chem in gene2chem[gene]:
            dc_inferred.append([disease, str(chem), str(gene)])
            
dc_inferred = pd.DataFrame(dc_inferred, columns=['disease-id', 'pubchem-id', 'gene-id'])
dc_inferred = dc_inferred.groupby(['disease-id', 'pubchem-id']).agg(lambda s: '|'.join(s)).reset_index()
dc_inferred.head(2)

,disease-id,pubchem-id,gene-id
0,MESH:C000591739,10680,1588
1,MESH:C000591739,14403,1588


In [ ]:
# Merge
disease_chem_db = dc_inferred.merge(chem_dis_curated, on=['disease-id', 'pubchem-id'], how='outer').fillna('')
disease_chem_db.rename(columns={'gene-id': 'via-genes'}, inplace=True)
# del disease_chem_db['type']
del dc_inferred, chem_dis_curated

disease_chem_db.head(1)

,disease-id,pubchem-id,via-genes,type
0,MESH:C000591739,10680,1588,


In [ ]:
# disease_chem_db['weight'] = disease_chem_db['via-genes'].apply(lambda s: len(s.split('|')))

In [ ]:
disease_chem_db = disease_chem_db.where((pd.notnull(disease_chem_db)), None)

disease_chem_db.rename(columns={
    'pubchem-id': 'pubchem_id',
    'disease-id': 'disease_id',
    'type':'type_relation',
    'via-genes':'via_genes'
},inplace=True)

In [ ]:
disease_chem_db.to_csv(DBDUMP/'disease-chemical.tsv', sep='\t', encoding='utf-8', index=None)

In [ ]:
with app.app_context():
    add_to_db_df(Chemical_disease, disease_chem_db)

### Gene Chemical

In [ ]:
# Curated
chem_gene_curated = chem_gene.copy(deep=True)
chem_gene_curated['gene-id'] = chem_gene_curated['gene-id'].apply(str)
chem_gene_curated['pubchem-id'] = chem_gene_curated['pubchem-id'].apply(str)

# Inferred through diseases
gc_inferred = list()
for gene, dislst in gene2dis.items():
    for dis in dislst:
        for chem in dis2chem[dis]:
            gc_inferred.append([str(gene), str(chem), dis])
            
gc_inferred = pd.DataFrame(gc_inferred, columns=['gene-id', 'pubchem-id', 'disease-id'])
gc_inferred = gc_inferred.groupby(['gene-id', 'pubchem-id']).agg(lambda s: '|'.join(s)).reset_index()

In [ ]:
# Merge
chem_gene_db = gc_inferred.merge(chem_gene_curated, on=['gene-id', 'pubchem-id'], how='outer').fillna('')
chem_gene_db.rename(columns={'disease-id': 'via-diseases'}, inplace=True)
del chem_gene_db['source']
del gc_inferred, chem_gene_curated

In [ ]:
chem_gene_db = chem_gene_db.where((pd.notnull(chem_gene_db)), None)

chem_gene_db.rename(columns={
    'pubchem-id': 'pubchem_id',
    'gene-id': 'gene_id',
    'via-diseases': 'via_diseases',
    'interaction-actions': 'interaction_actions'
}, inplace=True)

In [ ]:
chem_gene_db = pd.read_csv(DATA / 'chemical-gene.tsv', sep='\t', encoding='utf-8', index_col=None)

In [ ]:
with app.app_context():
    add_to_db_df(Chemical_gene, chem_gene_db)

### Chemical Images

In [ ]:
from rdkit.Chem import MolFromSmiles, Draw
from rdkit.Chem.AllChem import Compute2DCoords
from shutil import copy

In [ ]:
!rm /dietrx/static/images/molecules/*

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
with app.app_context():
    for mol in Chemical.query.all():
        try:
            m = MolFromSmiles(mol.smiles)
            tmp = Compute2DCoords(m)
            Draw.MolToFile(m, 'static/images/molecules/' + str(mol.pubchem_id) + '.png')
        except:
            print(mol.mol_id, mol.smiles)
            copy('static/images/no-image.png', 
                'static/images/molecules/' + str(mol.pubchem_id) + '.png')

[16:01:53] unsupported number of radical electrons 4
[16:01:59] unsupported number of radical electrons 4
[16:02:22] WARNING: not removing hydrogen atom without neighbors
[16:02:22] WARNING: not removing hydrogen atom without neighbors
[16:02:22] WARNING: not removing hydrogen atom without neighbors
[16:02:22] WARNING: not removing hydrogen atom without neighbors
[16:02:22] WARNING: not removing hydrogen atom without neighbors
[16:02:22] WARNING: not removing hydrogen atom without neighbors
[16:02:22] WARNING: not removing hydrogen atom without neighbors
[16:02:22] WARNING: not removing hydrogen atom without neighbors
[16:02:23] WARNING: not removing hydrogen atom without neighbors
[16:02:23] WARNING: not removing hydrogen atom without neighbors
[16:02:23] WARNING: not removing hydrogen atom without neighbors
[16:02:23] WARNING: not removing hydrogen atom without neighbors
[16:02:23] WARNING: not removing hydrogen atom without neighbors
[16:02:23] WARNING: not removing hydrogen atom wi

### SDF Files

In [ ]:
import requests
covered = set(list(int(p.name[:-4]) for p in Path('static/sdf_files/').glob('*.sdf')))

In [ ]:
with app.app_context():
    for i, mol in enumerate(Chemical.query.all()):
        if mol.pubchem_id not in covered:
            r = requests.get('https://cactus.nci.nih.gov/chemical/structure/%s/file?format=sdf&get3d=True' % mol.smiles)
            with open('static/sdf_files/%i.sdf' % mol.pubchem_id, 'w') as f:
                f.write(r.text)
                covered.add(i)
            
        if i % 10 == 0:
            print("Completed: %i" %i)

Completed: 0
Completed: 10
Completed: 20
Completed: 30
Completed: 40
Completed: 50
Completed: 60
Completed: 70
Completed: 80
Completed: 90
Completed: 100
Completed: 110
Completed: 120
Completed: 130
Completed: 140
Completed: 150
Completed: 160
Completed: 170
Completed: 180
Completed: 190
Completed: 200
Completed: 210
Completed: 220
Completed: 230
Completed: 240
Completed: 250
Completed: 260
Completed: 270
Completed: 280
Completed: 290
Completed: 300
Completed: 310
Completed: 320
Completed: 330
Completed: 340
Completed: 350
Completed: 360
Completed: 370
Completed: 380
Completed: 390
Completed: 400
Completed: 410
Completed: 420
Completed: 430
Completed: 440
Completed: 450
Completed: 460
Completed: 470
Completed: 480
Completed: 490
Completed: 500
Completed: 510
Completed: 520
Completed: 530
Completed: 540
Completed: 550
Completed: 560
Completed: 570
Completed: 580
Completed: 590
Completed: 600
Completed: 610
Completed: 620
Completed: 630
Completed: 640
Completed: 650
Completed: 660
Comple

### Refrest Elastic Search

In [ ]:
app.elasticsearch.indices.delete(index='food', ignore=[400, 404])
app.elasticsearch.indices.delete(index='disease', ignore=[400, 404])
app.elasticsearch.indices.delete(index='gene', ignore=[400, 404])

with app.app_context():
    Food.reindex("food_id")
    Disease.reindex("disease_id")
    Gene.reindex("gene_id")

    app.elasticsearch.indices.delete(index='chemical', ignore=[400, 404])

    Chemical.reindex("pubchem_id")

C:\Users\Sneha\AppData\Local\Temp\ipykernel_9108\3898774582.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  app.elasticsearch.indices.delete(index='food', ignore=[400, 404])
C:\Users\Sneha\AppData\Local\Temp\ipykernel_9108\3898774582.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  app.elasticsearch.indices.delete(index='food', ignore=[400, 404])
C:\Users\Sneha\AppData\Local\Temp\ipykernel_9108\3898774582.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  app.elasticsearch.indices.delete(index='disease', ignore=[400, 404])
C:\Users\Sneha\AppData\Local\Temp\ipykernel_9108\3898774582.py:2: ElasticsearchWarning: Elasticsea

### Post processing

In [ ]:
with app.app_context():
    query = db.session.query(Food_disease.food_id.distinct().label("food_id"))
    food_ids = [row.food_id for row in query.all()]

In [ ]:
with app.app_context():
    query = db.session.query(Food_chemical.food_id.distinct().label("food_id"))
    food_ids += [row.food_id for row in query.all()]

In [ ]:
with app.app_context():
    query = db.session.query(Food_gene.food_id.distinct().label("food_id"))
    food_ids += [row.food_id for row in query.all()]

In [ ]:
food_ids = set(food_ids)

In [ ]:
with app.app_context():
    query = db.session.query(Food.food_id.distinct().label("food_id"))
    all_food_ids = set([row.food_id for row in query.all()])

In [ ]:
empty_ids = all_food_ids - food_ids

In [ ]:
with app.app_context():
    for fid in empty_ids:
        Food.query.filter_by(food_id=fid).delete()

In [ ]:
with app.app_context():
    db.session.commit()

In [ ]:
with app.app_context():
    for obj in Food.query.filter(Food.food_id.in_(list(all_food_ids - food_ids))).all():
        db.session.delete(obj)

In [ ]:
obj

<Food Cherax tenuimanus>

In [ ]:
with app.app_context():
    valid_food_ids = {f.food_id for f in db.session.query(Food.food_id).distinct()}
    all_food_ids = {fc.food_id for fc in db.session.query(Food_chemical.food_id).distinct()}
    renegade_ids = list(all_food_ids - valid_food_ids)


In [ ]:
with app.app_context():
    for obj in Food_chemical.query.filter(Food_chemical.food_id.in_(renegade_ids)).all():
        db.session.delete(obj)

In [ ]:
with app.app_context():
    db.session.commit()

In [ ]:
with app.app_context():
    results = db.session.query(Food_disease).join(Food).filter(Food.food_category == 'Seafood').all()

In [ ]:
with app.app_context():
    Food.query.filter_by(food_category='Seafood').delete()

In [ ]:
with app.app_context():
    db.session.commit()